In [ ]:
import pandas as pd
from utils import ROOT_DIR

In [ ]:
SEED = 671993
TRAIN_SAMPLED_PLAYLISTS = 300000

In [ ]:
from ast import literal_eval

df_artists = pd.read_csv(
    f"{ROOT_DIR}/data/external/artists.csv.gz",
    usecols=["artist_uri", "genres"]
)

# Filter artists with no genres
df_artists = df_artists[df_artists["genres"]!="[]"]

# Eval genres as list. Se puede hacer en read_csv (buen dato)
df_artists["genres"] = df_artists["genres"].apply(literal_eval)

In [ ]:
df_artists.genres.head(3)

In [ ]:
from collections import Counter

N_GENRES = 6000

def flatten(t):
    return [item for sublist in t for item in sublist]

# Lista de generos
genres_list = df_artists.genres.to_list()
genres_list = flatten(genres_list)

most_common_genres = Counter(genres_list).most_common(N_GENRES)
most_common_genres = [t[0] for t in most_common_genres]

In [ ]:
most_common_genres[:3]

In [ ]:
len(most_common_genres)

In [ ]:
# Generacion de matriz binaria de generos
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

df_artists = df_artists.join(
    pd.DataFrame.sparse.from_spmatrix(
        mlb.fit_transform(df_artists.pop('genres')),
        index=df_artists.index,
        columns=mlb.classes_
    )
)

In [ ]:
most_common_genres.insert(0, "artist_uri")
df_artists = df_artists[most_common_genres]

In [ ]:
df_artists.head(5)

In [ ]:
# Store artist genres into results
df_artists.to_pickle(f"{ROOT_DIR}/results/artists_profiles_all_genres.pickle")
